In [1032]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings

import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

In [1033]:
df = pd.read_excel('IEEE compétition.xlsx')

In [1098]:
#Data Cleaning 
def Cleaning(df):
    
    df["Mobilité"] = df["Mobilité"].astype(str)
    df = df[df['age  patient '] <= 100]
    df = df.drop(columns=[" numero patient","sexe","age  patient "])
    df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
    #target 
    y = df['Les Urgences En MED DENT'] 
    #features
    x = df.drop(columns='Les Urgences En MED DENT')

    #douleur spontanee
    x['Douleur Spontanée'] = x['Douleur Spontanée'].replace('occassionnellement','occasionnelle')

    # Douleur Provoquée
    #nothing


    # Col Mobilité 
    x['Mobilité'] = x['Mobilité'].replace({'1': 'oui','1+': '1_plus'})



    # Col Vitalité
    x['Vitalité'] = x['Vitalité'].replace({'vivante': 'dent vivante',
                                           'oui': 'dent vivante'})

    # Col "" Palpation Apicale ""
    x['Palpation Apicale'] = x['Palpation Apicale'].replace({'négatif':'non',
                                                             'negatif':'non'})


    # Col "" 'Médication ATG ""
    x['Médication ATG'] = x['Médication ATG'].replace({'non': 'non efficace',
                                                       'oui': 'efficace'})

    # Douleur Spontanée

    x['Soulager '] = x['Soulager '].replace({'eviter la mastication':'sans contact', 'parfois par le froid':'froid'})

    # Col "" 'Odeur Fétide ""

    x['Médication ATG'] = x['Médication ATG'].replace('odeur fétide', 'oui')
   
    return x,y

In [1099]:
x,y = Cleaning(df)

In [1100]:

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=23,test_size=0.2)

In [1101]:
num=0
hist = []
le= LabelEncoder()

for col in (x.columns):

    x_train[col] =le.fit_transform(x_train[col])
    x_test[col] = le.transform(x_test[col])
    hist.append(le)            
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [1102]:
TreeModel = DecisionTreeClassifier()
TreeModel.fit(x_train, y_train)
scT = TreeModel.score(x_test, y_test)*100
print('score de la model Decision Tree est %.2f%%' %scT)

score de la model Decision Tree est 100.00%


In [1103]:
LogisticModel = LogisticRegression()
LogisticModel.fit(x_train, y_train)
scL = LogisticModel.score(x_test, y_test)*100
print('score de la model Logistic Regression est %.2f%%' %scL)

score de la model Logistic Regression est 100.00%


In [1104]:
nb = GaussianNB()
nb.fit(x_train, y_train)

scNB = nb.score(x_test, y_test)*100
print('score de la model GaussianNB est %.2f%%' %score)

score de la model GaussianNB est 100.00%


In [1105]:
voting_clf = VotingClassifier(estimators=[('lr', TreeModel), ('nb', LogisticModel), ('dt', nb)], voting='hard')
voting_clf.fit(x_train, y_train)
score_maj = voting_clf.score(x_test, y_test)*100
print(f"Score du classifieur de vote majoritaire : {score_maj:.2f}")

Score du classifieur de vote majoritaire : 100.00


In [1107]:
def preprocessing(x):
    categorical = x#.drop(columns="age  patient ")
    categorical_columns = categorical.columns
    num=0
    for col in (categorical_columns):
        le = hist[num]
        x[col] = le.transform(x[col])
        num+=1
    x = scaler.transform(x)
    return x

In [1108]:
def pred(test_dataset):
    x , y = Cleaning(test_dataset)
    x = preprocessing(x)
    
    return voting_clf.predict(x)
    

In [1109]:
sample = pd.DataFrame(df.iloc[15:20])
pred(sample)

ValueError: y contains previously unseen labels: 'irradiée'

In [1113]:
#testing one sample:
sample = pd.DataFrame(df.iloc[[15]])
x_sample,y_sample = Cleaning(sample)
print(x_sample)
x_sample = preprocessing(x_sample)

print('Model answer:',voting_clf.predict(x_sample))
print('Real target value: ', y_sample)


   Douleur Spontanée         Douleur Provoquée Mobilité      Vitalité  \
15          irradiée  parfois a la mastication      non  dent vivante   

   Palpation Apicale Fievre Asthenie Odeur Fétide  \
15               non    non      non          non   

                     Médication ATG Soulager   \
15  efficace , si stade avancée non     froid   

   Observations Exobuccales ( remarque generale )   
15                                             non  


ValueError: y contains previously unseen labels: 'irradiée'

In [1111]:
exemple = pd.DataFrame(df.iloc[[15]])
pred(exemple)

ValueError: y contains previously unseen labels: 'irradiée'

In [1070]:
exemple = df.iloc[15:20]
pred(exemple)

ValueError: y contains previously unseen labels: 'irradiée'